# 没有验证集的

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score,f1_score,precision_score,recall_score,accuracy_score
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression, Ridge
import lightgbm
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def cv(train, test, target,kfold = None):
    test_preds = np.zeros((len(test)))
    cv = 0
    kf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=42)
    train_targets = train[target]
    train = train.drop(columns = [target])
    
    auc_xgb = 0
    auc_cat = 0
    auc_lgb = 0
    
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train, train_targets)):
        print(f"------------> Fold {fold + 1} <-----------------")
        X_train, X_valid = train.iloc[train_idx], train.iloc[valid_idx]
        y_train, y_valid = train_targets.iloc[train_idx], train_targets.iloc[valid_idx]
        class_weights = [1, len(y_train[y_train == 0]) / len(y_train[y_train == 1])]

        ctb = CatBoostClassifier(n_estimators=10000,
                                 early_stopping_rounds=100,
                                 class_weights=class_weights,
                                 **{'depth': 4, 'subsample': 0.9037951675853159, 'l2_leaf_reg': 0.1,
                                    'learning_rate': 0.33153188532829714, "thread_count": -1,
                                    'loss_function': 'Logloss', 'bootstrap_type': 'Bernoulli'})

        ctb.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=0)

        ctb_preds = ctb.predict(X_valid)
        ctb_test_preds = ctb.predict(test)
        
        auc_cat +=roc_auc_score(y_valid, ctb_preds)
        print(f"|------------------------------------|")
        print(f"|                                    |")
        print(f"|            CatBoost                |")
        print(f"|                                    |")
        print(f"|------------------------------------|")
        print(" ")
        print(f"------------- Train -----------------")
        print(f"F1 Score: {f1_score(y_valid, ctb_preds)}")
        print(f"F1-macro Score: {f1_score(y_valid, ctb_preds, average='macro')}")
        print(f"F1-micro Score: {f1_score(y_valid, ctb_preds, average='micro')}")
        print(f"Accuracy: {accuracy_score(y_valid, ctb_preds)}")
        print(f"Precision: {precision_score(y_valid, ctb_preds)}")
        print(f"Recall: {recall_score(y_valid, ctb_preds)}")
        print(f"AUC: {roc_auc_score(y_valid, ctb_preds)}")
        print(" ")

        lgb = LGBMClassifier(n_estimators=10000,
                             early_stopping_rounds=100,
                             class_weight='balanced',
                             **{'max_depth': 6,
                                'subsample': 0.5528027451731908,
                                'colsample_bytree': 1,
                                'learning_rate': 0.11430358849370452,
                                'bagging_freq': 5,
                                'bagging_fraction': 0.6,
                                'boost_from_average': 'false',
                                'boosting_type': 'gbdt',
                                'feature_fraction': 0.5,
                                'min_data_in_leaf': 100,
                                'min_sum_hessian_in_leaf': 100.0,
                                'num_leaves': 2 ** 4,
                                'objective': 'binary', 'n_jobs': -1})

        lgb.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=0)

        lgb_preds = lgb.predict(X_valid)
        lgb_test_preds = lgb.predict(test)
        
        auc_lgb +=roc_auc_score(y_valid, lgb_preds)
        print(f"|------------------------------------|")
        print(f"|                                    |")
        print(f"|            LightGBM                |")
        print(f"|                                    |")
        print(f"|------------------------------------|")
        print(" ")
        print(f"------------- Train -----------------")
        print(f"F1 Score: {f1_score(y_valid, lgb_preds)}")
        print(f"F1-macro Score: {f1_score(y_valid, lgb_preds, average='macro')}")
        print(f"F1-micro Score: {f1_score(y_valid, lgb_preds, average='micro')}")
        print(f"Accuracy: {accuracy_score(y_valid, lgb_preds)}")
        print(f"Precision: {precision_score(y_valid, lgb_preds)}")
        print(f"Recall: {recall_score(y_valid, lgb_preds)}")
        print(f"AUC: {roc_auc_score(y_valid, lgb_preds)}")

        xgb = XGBClassifier(n_estimators=10000,
                            early_stopping_rounds=100,
                            **{'max_depth': 6, 'subsample': 0.5549531235327668,
                               'scale_pos_weight':sum(y_train == 0) / sum(y_train == 1),
                               'colsample_bytree': 1,
                               'learning_rate': 0.15, 'objective': 'binary:logistic', 'n_jobs': -1})

        xgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)

        xgb_preds = xgb.predict(X_valid)
        xgb_test_preds = xgb.predict(test)
        
        auc_xgb +=roc_auc_score(y_valid, xgb_preds)
        print(f"|------------------------------------|")
        print(f"|                                    |")
        print(f"|            XGBoost                 |")
        print(f"|                                    |")
        print(f"|------------------------------------|")
        print(" ")
        print(f"------------- Train -----------------")
        print(f"F1 Score: {f1_score(y_valid, xgb_preds)}")
        print(f"F1-macro Score: {f1_score(y_valid, xgb_preds, average='macro')}")
        print(f"F1-micro Score: {f1_score(y_valid, xgb_preds, average='micro')}")
        print(f"Accuracy: {accuracy_score(y_valid, xgb_preds)}")
        print(f"Precision: {precision_score(y_valid, xgb_preds)}")
        print(f"Recall: {recall_score(y_valid, xgb_preds)}")
        print(f"AUC: {roc_auc_score(y_valid, xgb_preds)}")

        meta_train = [lgb_preds, ctb_preds, xgb_preds]
        meta_test = [lgb_test_preds, ctb_test_preds, xgb_test_preds]
    print(f"cat Auc average: {auc_cat/kfold}")
    print(f"lgb Auc average: {auc_lgb/kfold}")
    print(f"xgb Auc average: {auc_xgb/kfold}")
    print(f"CV: {cv/(fold+1)}")
    
    return meta_train,meta_test

_ , pred_test = cv(train = train, test = test.drop(columns = ['Severity']), target = 'Severity',kfold=5)


In [ ]:
def test_score(y_true,y_pred):
    print(f"F1 Score: {f1_score(y_true, y_pred)}")
    print(f"F1-macro Score: {f1_score(y_true, y_pred, average='macro')}")
    print(f"F1-micro Score: {f1_score(y_true, y_pred, average='micro')}")
    print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
    print(f"Precision: {precision_score(y_true, y_pred)}")
    print(f"Recall: {recall_score(y_true, y_pred)}")
    print(f"AUC: {roc_auc_score(y_true, y_pred)}")
    
for i in range(3):
    print("-"*20)
    test_score(test['Severity'],pred_test[i])


# 带有验证集的

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score,f1_score,precision_score,recall_score,accuracy_score
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression, Ridge
import lightgbm
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def cv_withVal(train, val, test, target):
    test_preds = np.zeros((len(test)))
    val_preds = np.zeros((len(val)))
    cv = 0
    kf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
    train_targets = train[target]
    train = train.drop(columns = [target])
    
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train, train_targets)):
        print("-" * 10 + ">" + f"Fold {fold + 1}" + "<" + "-" * 10)
        X_train, X_valid = train.iloc[train_idx], train.iloc[valid_idx]
        y_train, y_valid = train_targets.iloc[train_idx], train_targets.iloc[valid_idx]
        class_weights = [1, len(y_train[y_train == 0]) / len(y_train[y_train == 1])]

        ctb = CatBoostClassifier(n_estimators=10000,
                                 early_stopping_rounds=100,
                                 class_weights=class_weights,
                                 **{'depth': 4, 'subsample': 0.9037951675853159, 'l2_leaf_reg': 4.085968446512874,
                                    'learning_rate': 0.33153188532829714, "thread_count": -1,
                                    'loss_function': 'Logloss', 'bootstrap_type': 'Bernoulli'})

        ctb.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=0)

        ctb_preds = ctb.predict(X_valid)
        ctb_val_preds = ctb.predict(val.drop(columns = [target]))
        ctb_test_preds = ctb.predict(test)

        print(f"------------- Train -----------------")
        print(f"F1 Score: {f1_score(y_valid, ctb_preds)}")
        print(f"F1-macro Score: {f1_score(y_valid, ctb_preds, average='macro')}")
        print(f"F1-micro Score: {f1_score(y_valid, ctb_preds, average='micro')}")
        print(f"Accuracy: {accuracy_score(y_valid, ctb_preds)}")
        print(f"Precision: {precision_score(y_valid, ctb_preds)}")
        print(f"Recall: {recall_score(y_valid, ctb_preds)}")
        print(f"AUC: {roc_auc_score(y_valid, ctb_preds)}")

        print(f"------------- val -----------------")
        print(f"F1 Score: {f1_score(y_val, ctb_val_preds)}")
        print(f"F1-macro Score: {f1_score(y_val, ctb_val_preds, average='macro')}")
        print(f"F1-micro Score: {f1_score(y_val, ctb_val_preds, average='micro')}")
        print(f"Accuracy: {accuracy_score(y_val, ctb_val_preds)}")
        print(f"Precision: {precision_score(y_val, ctb_val_preds)}")
        print(f"Recall: {recall_score(y_val, ctb_val_preds)}")
        print(f"AUC: {roc_auc_score(y_val, ctb_val_preds)}")

        print(" ")

        lgb = LGBMClassifier(n_estimators=10000,
                             early_stopping_rounds=100,
                             class_weight='balanced',
                             **{'max_depth': 5,
                                'subsample': 0.5528027451731908,
                                'colsample_bytree': 0.26714040676512085,
                                'learning_rate': 0.11430358849370452,
                                'bagging_freq': 5,
                                'bagging_fraction': 0.6,
                                'boost_from_average': 'false',
                                'boosting_type': 'gbdt',
                                'feature_fraction': 0.5,
                                'min_data_in_leaf': 100,
                                'min_sum_hessian_in_leaf': 100.0,
                                'num_leaves': 2 ** 4,
                                'objective': 'binary', 'n_jobs': -1})

        lgb.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=0)

        lgb_preds = lgb.predict(X_valid)
        lgb_val_preds = lgb.predict(val.drop(columns = [target]))
        lgb_test_preds = lgb.predict(test)

        print(f"------------- Train -----------------")
        print(f"F1 Score: {f1_score(y_valid, lgb_preds)}")
        print(f"F1-macro Score: {f1_score(y_valid, lgb_preds, average='macro')}")
        print(f"F1-micro Score: {f1_score(y_valid, lgb_preds, average='micro')}")
        print(f"Accuracy: {accuracy_score(y_valid, lgb_preds)}")
        print(f"Precision: {precision_score(y_valid, lgb_preds)}")
        print(f"Recall: {recall_score(y_valid, lgb_preds)}")
        print(f"AUC: {roc_auc_score(y_valid, lgb_preds)}")

        print(f"------------- val -----------------")
        print(f"F1 Score: {f1_score(y_val, lgb_val_preds)}")
        print(f"F1-macro Score: {f1_score(y_val, lgb_val_preds, average='macro')}")
        print(f"F1-micro Score: {f1_score(y_val, lgb_val_preds, average='micro')}")
        print(f"Accuracy: {accuracy_score(y_val, lgb_val_preds)}")
        print(f"Precision: {precision_score(y_val, lgb_val_preds)}")
        print(f"Recall: {recall_score(y_val, lgb_val_preds)}")
        print(f"AUC: {roc_auc_score(y_val, lgb_val_preds)}")
        print(" ")

        xgb = XGBClassifier(n_estimators=10000,
                            early_stopping_rounds=100,
                            **{'max_depth': 6, 'subsample': 0.5549531235327668,
                               'scale_pos_weight':sum(y_train == 0) / sum(y_train == 1),
                               'colsample_bytree': 0.20335345531973037,
                               'learning_rate': 0.027251718255125106, 'objective': 'binary:logistic', 'n_jobs': -1})

        xgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)

        xgb_preds = xgb.predict(X_valid)
        xgb_val_preds = xgb.predict(val.drop(columns = [target]))
        xgb_test_preds = xgb.predict(test)

        print(f"------------- Train -----------------")
        print(f"F1 Score: {f1_score(y_valid, xgb_preds)}")
        print(f"F1-macro Score: {f1_score(y_valid, xgb_preds, average='macro')}")
        print(f"F1-micro Score: {f1_score(y_valid, xgb_preds, average='micro')}")
        print(f"Accuracy: {accuracy_score(y_valid, xgb_preds)}")
        print(f"Precision: {precision_score(y_valid, xgb_preds)}")
        print(f"Recall: {recall_score(y_valid, xgb_preds)}")
        print(f"AUC: {roc_auc_score(y_valid, xgb_preds)}")

        print(f"------------- val -----------------")
        print(f"F1 Score: {f1_score(y_val, xgb_val_preds)}")
        print(f"F1-macro Score: {f1_score(y_val, xgb_val_preds, average='macro')}")
        print(f"F1-micro Score: {f1_score(y_val, xgb_val_preds, average='micro')}")
        print(f"Accuracy: {accuracy_score(y_val, xgb_val_preds)}")
        print(f"Precision: {precision_score(y_val, xgb_val_preds)}")
        print(f"Recall: {recall_score(y_val, xgb_val_preds)}")
        print(f"AUC: {roc_auc_score(y_val, xgb_val_preds)}")
        print(" ")

        meta_train = [lgb_preds, ctb_preds, xgb_preds]
        meta_test = [lgb_test_preds, ctb_test_preds, xgb_test_preds]
        meta_val = [lgb_val_preds, ctb_val_preds, xgb_val_preds]

    print(f"CV: {cv/(fold+1)}")
    
    return meta_train, meta_val, meta_test

_ , pred_val, pred_test = cv(train = pd.concat([x_train,y_train],axis =1).reset_index(drop=True), val = pd.concat([x_val,y_val],axis =1).reset_index(drop=True), 
                             test = test.drop(columns = ['Severity']), target = 'Severity')
